In [2]:
# import

%matplotlib inline

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import geopy.distance as distance


import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import json


In [ ]:
## RAW DATA - no need to read

# load wapo
wapo_df = pd.read_csv('./data/fatal-police-shootings-data.csv')

# load MPV data
mpv_df = pd.read_excel(pd.ExcelFile('data/MPVDatasetDownload_CleanColumnNames.xlsx'), sheet_name='2013-2020 Police Killings')

# load county-wise demographic info from US census
county_df = pd.read_csv('data/acs2015_county_data.csv')

# load election data from HW 1
election_county_df = pd.read_csv('data/election2016_by_county.csv')

# lemas geocode
lemas_geocode = pd.read_csv('data/lemas_geocode.csv')

# load LEMAS
lemas_raw = pd.read_csv('data/LEMAS_2016.tsv', sep = '\t', index_col = 'LEAR_ID', header = 0, encoding = 'unicode_escape')

# rename some variables for merging - eg. merge county demographic info into LEMAS using FIPS
county_df['FIPS'] = county_df.CensusId
election_county_df['FIPS'] = election_county_df.fipscode


In [5]:
# read in clean victim list with matched ORI9 codes for each 

vic_df = pd.read_csv('data/victim_data_with_clean_ori9code.csv')

# load LEMAS
lemas_raw = pd.read_csv('data/LEMAS_2016.tsv', sep = '\t', index_col = 'LEAR_ID', header = 0, encoding = 'unicode_escape')
vic_df.head()

In [9]:
# take a subset of lemas data:

# extract relevant stats for LEMAS - MORE SHOULD BE DONE HERE
def clean_lemas(lemas_raw):
    race_fields = ['WHITE','BLACK','HISP','AMIND','ASIAN','HAWPI','MULTI','UNK']
    gender_fields = ['MALE','FEM']

    # remove rows that do not have demographic info
    #lemas_raw = lemas_raw[(lemas_raw.PERS_MALE>=0) & (lemas_raw.PERS_FEMALE>=0) & (lemas_raw.PERS_WHITE_FEM>=0) & (lemas_raw.PERS_WHITE_MALE>=0)][:]


    lemas_raw['PERS_TOTAL'] = lemas_raw.PERS_MALE + lemas_raw.PERS_FEMALE

    # for now, take a fraction of the columns or it is overwhelming...
    # EDIT:
    lemas_new = lemas_raw.iloc[:,0:10]

    lemas_new['PERS_TOTAL'] = lemas_raw.PERS_MALE + lemas_raw.PERS_FEMALE
    
    for this_race in race_fields:
        lemas_new['PERS_' + this_race] = lemas_raw['PERS_' + this_race + '_MALE'] + lemas_raw['PERS_' + this_race + '_FEM']
        lemas_new['FRAC_' + this_race] = lemas_new['PERS_' + this_race]/lemas_new.PERS_TOTAL
    

    # make LEMAS FIPS CODE NUMERIC for merging with demographic data 
#     lemas_new.FIPS = pd.to_numeric(lemas_new.FIPS,errors='coerce')
#     lemas_new = lemas_new.dropna()
    
    return lemas_new


lemas_df = clean_lemas(lemas_raw)

lemas_df.head()

,AGENCYNAME,CITY,ZIPCODE,STATE,COUNTY,FIPS,ORI9,POPSERVED,POPGROUP,AGENCYTYPE,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
LEAR_ID,,,,,,,,,,,,,,,,,,,,,
635592,DAVIS POLICE DEPARTMENT,DAVIS,95618,CA,YOLO,6113,CA0570100,68111,5,1,...,0,0.0,6,0.1,0,0.0,0,0.000000,0,0.0
645110,WEST NEW YORK POLICE,WEST NEW YORK,7093,NJ,HUDSON,34017,NJ0091200,53343,5,1,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0
631270,WESTOVER POLICE DEPARTMENT,WESTOVER,26501,WV,MONONGALIA,54061,WV0310400,4243,8,1,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0
631316,BARABOO POLICE DEPARTMENT,BARABOO,53913,WI,SAUK,55111,WI0570200,12173,7,1,...,0,0.0,0,0.0,0,0.0,1,0.034483,0,0.0
631684,OREGON POLICE DEPARTMENT,OREGON,53575,WI,DANE,55025,WI0137400,3334,8,1,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0


In [10]:
# merge lemas 

vic_df_lemas = vic_df.merge(lemas_df,how='left',on='ORI9',suffixes=['_wapo','_lemas'])

vic_df_lemas.head()

,Unnamed: 0,id,name,date,armed_wapo,age,gender,race,city_wapo,state,...,PERS_AMIND,FRAC_AMIND,PERS_ASIAN,FRAC_ASIAN,PERS_HAWPI,FRAC_HAWPI,PERS_MULTI,FRAC_MULTI,PERS_UNK,FRAC_UNK
0,0,3,Tim Elliot,2015-01-02,gun,53.0,M,A,Shelton,WA,...,3,0.055556,0,0.000000,0,0.000000,0,0.0,0,0.000000
1,1,4,Lewis Lee Lembke,2015-01-02,gun,47.0,M,W,Aloha,OR,...,3,0.007463,16,0.039801,0,0.000000,0,0.0,3,0.007463
2,2,5,John Paul Quintero,2015-01-03,unarmed,23.0,M,H,Wichita,KS,...,2,0.002950,21,0.030973,0,0.000000,0,0.0,0,0.000000
3,3,8,Matthew Hoffman,2015-01-04,toy weapon,32.0,M,W,San Francisco,CA,...,7,0.002971,385,0.163413,140,0.059423,0,0.0,38,0.016129
4,4,9,Michael Rodriguez,2015-01-04,nail gun,39.0,M,H,Evans,CO,...,0,0.000000,0,0.000000,0,0.000000,0,0.0,0,0.000000
